In [145]:
# Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import seaborn as sns
import datetime
import os
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder


In [200]:
# Load data
path = os.getcwd()
df = pd.read_csv(path+"/train.csv")
# Working with the train data first then with test data. Discuss whether to combine both when doing exploration? 
# dt = pd.read_csv(path+"/test.csv")
# Shape of train data 
print('df train', df.shape)

df.head()

df train (6895, 70)


,id,short_name,overall,potential,value_eur,wage_eur,birthday_date,height_cm,weight_kg,club_name,...,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,position
0,216302,E. García,71,71,1400000.0,10000,1989-12-28,176,73,Club Atlético de San Luis,...,65,66,65,14,11,12,12,12,NaN,LB
1,237867,D. Cancola,65,71,1000000.0,2000,1996-10-23,183,73,Ross County FC,...,65,61,58,10,13,7,6,11,NaN,LDM
2,253472,E. Kahl,65,77,1600000.0,2000,2001-09-27,178,69,Aarhus GF,...,60,58,59,10,10,8,10,11,NaN,LWB
3,223994,S. Mugoša,72,72,2300000.0,5000,1992-02-26,188,81,Incheon United FC,...,16,22,19,16,15,13,8,9,NaN,LS
4,251635,A. Țigănașu,65,65,525000.0,3000,1990-06-12,179,74,FC Botoşani,...,64,61,58,12,5,11,12,15,NaN,LB


In [201]:
df.columns

Index(['id', 'short_name', 'overall', 'potential', 'value_eur', 'wage_eur',
       'birthday_date', 'height_cm', 'weight_kg', 'club_name', 'league_name',
       'league_level', 'club_jersey_number', 'club_loaned_from', 'club_joined',
       'club_contract_valid_until', 'nationality_name', 'nation_jersey_number',
       'preferred_foot', 'weak_foot', 'skill_moves',
       'international_reputation', 'work_rate', 'body_type', 'real_face',
       'release_clause_eur', 'player_tags', 'player_traits', 'pace',
       'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'pow

In [ ]:
# Checking if a variable is numerical or categorical 
numerical_vars = []
categorical_vars = []

for var in df.columns:
    
    
    numerical_bool = ('numerical' if df[var].dtype in ['int64', 'float64'] else 'categorical')
    
    if numerical_bool == 'numerical':
        numerical_vars.append(var)
    elif numerical_bool == 'categorical':
        categorical_vars.append(var)
    else:
        print('ERROR')
    
    #print(f"> {var}: is {numerical_bool}")

#Numerical List & Categorical List
display(pd.DataFrame({'numerical variables': numerical_vars}, index = range(1, len(numerical_vars)+1)))
display(pd.DataFrame({'categorical variables': categorical_vars}, index = range(1, len(categorical_vars)+1)))

In [202]:
# Function calculating missing values by column
def calculate_column_missing_percentage(df):
    total_rows = len(df)
    missing_percentages = (df.isnull().sum() / total_rows) * 100
    return missing_percentages

# Missing values in df
missing_percentages = calculate_column_missing_percentage(df)

# Display the columns with missing values only
missing_percentages_sorted = missing_percentages[missing_percentages > 0].sort_values(ascending=False)
display(missing_percentages_sorted)

# dropping the colimns with a high percentage of missing values 
columns_to_drop = ['club_loaned_from', 'nation_jersey_number', 'goalkeeping_speed', 'player_tags']
df = df.drop(columns=columns_to_drop)


club_loaned_from        94.909355
nation_jersey_number    94.010152
goalkeeping_speed       90.848441
player_tags             88.817984
player_traits           37.548949
pace                     9.151559
shooting                 9.151559
passing                  9.151559
dribbling                9.151559
defending                9.151559
physic                   9.151559
release_clause_eur       5.119652
club_joined              5.090645
value_eur                0.029007
dtype: float64

Age could be important factor get from birthdate and then create two columns with year and month deleting the day cause is not as relevant. 

In [203]:
# Calculate age of players
df['birthday_date'] = pd.to_datetime(df['birthday_date'], format='%Y-%m-%d', errors='coerce')
df['today'] = pd.to_datetime('2022-11-01',format='%Y-%m-%d', errors='coerce')


df['age'] = ((df['today'] - df['birthday_date'])/np.timedelta64(1, 'Y'))
df['age'] = df['age'].astype(int)

In [204]:

# Extract the month and year into separate columns
# Dropping the day because does not seem as relevant as a month or year a player is born 
df['BirthMonth'] = df['birthday_date'].dt.month
df['BirthYear'] = df['birthday_date'].dt.year

# Drop the original 'birthday_date' column 
df = df.drop('birthday_date', axis=1)
# Display the resulting dataframe
display(df)


,id,short_name,overall,potential,value_eur,wage_eur,height_cm,weight_kg,club_name,league_name,...,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,position,today,age,BirthMonth,BirthYear
0,216302,E. García,71,71,1400000.0,10000,176,73,Club Atlético de San Luis,Mexican Liga MX,...,14,11,12,12,12,LB,2022-11-01,32,12,1989
1,237867,D. Cancola,65,71,1000000.0,2000,183,73,Ross County FC,Scottish Premiership,...,10,13,7,6,11,LDM,2022-11-01,26,10,1996
2,253472,E. Kahl,65,77,1600000.0,2000,178,69,Aarhus GF,Danish Superliga,...,10,10,8,10,11,LWB,2022-11-01,21,9,2001
3,223994,S. Mugoša,72,72,2300000.0,5000,188,81,Incheon United FC,Korean K League 1,...,16,15,13,8,9,LS,2022-11-01,30,2,1992
4,251635,A. Țigănașu,65,65,525000.0,3000,179,74,FC Botoşani,Romanian Liga I,...,12,5,11,12,15,LB,2022-11-01,32,6,1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6890,242007,C. Whelan,59,66,425000.0,2000,180,65,Carlisle United,English League Two,...,12,9,10,8,13,RCB,2022-11-01,24,12,1997
6891,187961,Paulinho,83,83,28500000.0,61000,183,80,Al Ahli,Saudi Abdul L. Jameel League,...,16,16,11,6,8,LCM,2022-11-01,34,7,1988
6892,257234,Y. Hamache,70,80,3400000.0,6000,177,73,Boavista FC,Portuguese Liga ZON SAGRES,...,8,9,5,8,10,LWB,2022-11-01,23,7,1999
6893,232511,S. Sasaki,71,71,1300000.0,7000,176,70,Sanfrecce Hiroshima,Japanese J. League Division 1,...,8,10,10,10,8,LCB,2022-11-01,33,10,1989


In [205]:
# Target variable 
df.position.describe()


count     6895
unique      24
top        RCB
freq       631
Name: position, dtype: object

In [206]:
df.position.unique()

array(['LB', 'LDM', 'LWB', 'LS', 'RW', 'GK', 'LCM', 'RM', 'RCM', 'LW',
       'LCB', 'RB', 'RCB', 'CDM', 'RDM', 'CAM', 'CB', 'ST', 'RS', 'LM',
       'RWB', 'LF', 'RF', 'CM'], dtype=object)

<p> Things to potentially remove: 
<ol>
    <li> short_name 
<ol>

<p> Things removed already because they have a lot of NaN:
<ol>
    <li>  club_loaned_from
    <li> nation_jersey_number 
    <li> goalkeeping_speed 
    <li> player_tags 
<ol>


Things to consider for position: 
<ol>
    <li> overall 
    <li> potential 
    <li> value_eur 
    <li> wage_eur 
    <li> birthday_date (AGE) 
    <li> height_cm 
    <li> weight_kg 
    <li> league_level 
    <li> club_joined 
    <li> weak_foot 
    <li> skill_moves
    <li> preferred_foot
    <li> work_rate
    <li> body_type 
    <li> player_traits 
    <li> pace 
    <li> shooting 
    <li> passing 
    <li> dribbling 
    <li> defending 
    <li> physic 
    <li> attacking_crossing 
    <li> attacking_finishing
    <li> attacking_heading_accuracy 
    <li> attacking_short_passing 
    <li> attacking_volleys 
    <li> skill_dribbling
    <li> skill_curve 
    <li> skill_fk_accuracy
    <li> skill_long_passing 
    <li> skill_ball_control 
    <li> movement_acceleration 
    <li> movement_sprint_speed 
    <li> movement_agility 
    <li> movement_reactions 
    <li> movement_balance
    <li> power_shot_power 
    <li> power_jumping 
    <li> power_stamina 
    <li> power_strength 
    <li> power_long_shots
    <li> mentality_aggression 
    <li> mentality_interceptions 
    <li> mentality_positioning 
    <li> mentality_vision 
    <li> mentality_penalties 
    <li> mentality_composure 
    <li> defending_marking_awareness 
    <li> defending_standing_tackle 
    <li> defending_sliding_tackle 
    <li> goalkeeping_diving 
    <li> goalkeeping_handling 
    <li> goalkeeping_kicking 
    <li> goalkeeping_positioning 
    <li> goalkeeping_reflexes 
    <li> goalkeeping_speed
<ol>


Things not sure about: 
<ol>
    <li> club_name 
    <li> league_name
    <li> club_jersey_number 
    <li> club_contract_valid_until 
    <li>  nationality_name 
    <li>  international_reputation 
    <li> real_face 
    <li> release_clause_eur 
<ol>

In [207]:
# Checking for categorical variables
import pandas as pd

# Assuming 'df' is your DataFrame
categorical_columns = list(df.select_dtypes(include=['object']).columns)

# Display the categorical columns
print("Categorical Columns:", categorical_columns)


Categorical Columns: ['short_name', 'club_name', 'league_name', 'club_joined', 'nationality_name', 'preferred_foot', 'work_rate', 'body_type', 'real_face', 'player_traits', 'position']


Potentially drop real_face because not sure what it is?

In [208]:
# Look into some of the categories and see to label encode them 
import pandas as pd

# Assuming 'df' is your DataFrame and 'categorical_columns' is a list of categorical column names
categorical_columns = ['body_type', 'work_rate', 'real_face', 'position']  

# Create an empty dictionary to store DataFrames
categorical_counts_dict = {}

# Loop through categorical columns
for category in categorical_columns:
    # Count occurrences for each category
    category_counts = df[category].value_counts(ascending=False)
    
    # Create a DataFrame and store it in the dictionary
    category_counts_df = category_counts.reset_index()
    category_counts_df.columns = [category, 'Count']
    categorical_counts_dict[category] = category_counts_df

# Display the DataFrames
for category, counts_df in categorical_counts_dict.items():
    print(f"Club Counts DataFrame for {category}:")
    display(counts_df)

Club Counts DataFrame for body_type:


,body_type,Count
0,Normal (170-185),2471
1,Normal (185+),1630
2,Lean (170-185),1280
3,Lean (185+),628
4,Normal (170-),247
5,Stocky (170-185),228
6,Stocky (185+),145
7,Lean (170-),134
8,Unique,87
9,Stocky (170-),45


Club Counts DataFrame for work_rate:


,work_rate,Count
0,Medium/Medium,3125
1,High/Medium,1456
2,Medium/High,856
3,High/High,535
4,High/Low,299
5,Medium/Low,246
6,Low/High,197
7,Low/Medium,168
8,Low/Low,13


Club Counts DataFrame for real_face:


,real_face,Count
0,No,5849
1,Yes,1046


Club Counts DataFrame for position:


,position,Count
0,RCB,631
1,GK,631
2,LCB,631
3,RB,463
4,LB,463
5,ST,428
6,RCM,423
7,LCM,423
8,RM,369
9,LM,369


In [209]:
# Label Encode body_type 
categorical_columns_encode1 = ['body_type', 'work_rate', 'real_face']
label_encoders = {}

# Loop through categorical columns
for category in categorical_columns:
    # Initialize LabelEncoder for each category
    encoder = LabelEncoder()
    
    # Fit and transform the current column
    df[category + '_encoded'] = encoder.fit_transform(df[category])
    
    # Store the LabelEncoder in the dictionary
    label_encoders[category] = encoder

# Display the DataFrame with encoded columns
print("DataFrame with Encoded Columns:")
display(df)

DataFrame with Encoded Columns:


,id,short_name,overall,potential,value_eur,wage_eur,height_cm,weight_kg,club_name,league_name,...,goalkeeping_reflexes,position,today,age,BirthMonth,BirthYear,body_type_encoded,work_rate_encoded,real_face_encoded,position_encoded
0,216302,E. García,71,71,1400000.0,10000,176,73,Club Atlético de San Luis,Mexican Liga MX,...,12,LB,2022-11-01,32,12,1989,4,6,0,5
1,237867,D. Cancola,65,71,1000000.0,2000,183,73,Ross County FC,Scottish Premiership,...,11,LDM,2022-11-01,26,10,1996,1,8,0,8
2,253472,E. Kahl,65,77,1600000.0,2000,178,69,Aarhus GF,Danish Superliga,...,11,LWB,2022-11-01,21,9,2001,4,2,0,13
3,223994,S. Mugoša,72,72,2300000.0,5000,188,81,Incheon United FC,Korean K League 1,...,9,LS,2022-11-01,30,2,1992,2,2,0,11
4,251635,A. Țigănașu,65,65,525000.0,3000,179,74,FC Botoşani,Romanian Liga I,...,15,LB,2022-11-01,32,6,1990,4,6,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6890,242007,C. Whelan,59,66,425000.0,2000,180,65,Carlisle United,English League Two,...,13,RCB,2022-11-01,24,12,1997,4,8,0,15
6891,187961,Paulinho,83,83,28500000.0,61000,183,80,Al Ahli,Saudi Abdul L. Jameel League,...,8,LCM,2022-11-01,34,7,1988,4,0,1,7
6892,257234,Y. Hamache,70,80,3400000.0,6000,177,73,Boavista FC,Portuguese Liga ZON SAGRES,...,10,LWB,2022-11-01,23,7,1999,4,2,0,13
6893,232511,S. Sasaki,71,71,1300000.0,7000,176,70,Sanfrecce Hiroshima,Japanese J. League Division 1,...,8,LCB,2022-11-01,33,10,1989,1,6,0,6


In [210]:
df.club_joined.describe()

count           6544
unique          1263
top       2021-07-01
freq             449
Name: club_joined, dtype: object

In [214]:

# Extract the column with missing values
imputation_data = df[['club_joined']]

# Initialize KNNImputer
imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors

# Fit and transform the data to impute missing values
imputed_data = imputer.fit_transform(imputation_data)

# Replace the original column with the imputed values
df['club_joined'] = imputed_data

# Display the DataFrame with imputed values
print("DataFrame with Imputed Values:")
display(df)



DataFrame with Imputed Values:


,id,short_name,overall,potential,value_eur,wage_eur,height_cm,weight_kg,club_name,league_name,...,goalkeeping_reflexes,position,today,age,BirthMonth,BirthYear,body_type_encoded,work_rate_encoded,real_face_encoded,position_encoded
0,216302,E. García,71,71,1400000.0,10000,176,73,Club Atlético de San Luis,Mexican Liga MX,...,12,LB,2022-11-01,32,12,1989,4,6,0,5
1,237867,D. Cancola,65,71,1000000.0,2000,183,73,Ross County FC,Scottish Premiership,...,11,LDM,2022-11-01,26,10,1996,1,8,0,8
2,253472,E. Kahl,65,77,1600000.0,2000,178,69,Aarhus GF,Danish Superliga,...,11,LWB,2022-11-01,21,9,2001,4,2,0,13
3,223994,S. Mugoša,72,72,2300000.0,5000,188,81,Incheon United FC,Korean K League 1,...,9,LS,2022-11-01,30,2,1992,2,2,0,11
4,251635,A. Țigănașu,65,65,525000.0,3000,179,74,FC Botoşani,Romanian Liga I,...,15,LB,2022-11-01,32,6,1990,4,6,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6890,242007,C. Whelan,59,66,425000.0,2000,180,65,Carlisle United,English League Two,...,13,RCB,2022-11-01,24,12,1997,4,8,0,15
6891,187961,Paulinho,83,83,28500000.0,61000,183,80,Al Ahli,Saudi Abdul L. Jameel League,...,8,LCM,2022-11-01,34,7,1988,4,0,1,7
6892,257234,Y. Hamache,70,80,3400000.0,6000,177,73,Boavista FC,Portuguese Liga ZON SAGRES,...,10,LWB,2022-11-01,23,7,1999,4,2,0,13
6893,232511,S. Sasaki,71,71,1300000.0,7000,176,70,Sanfrecce Hiroshima,Japanese J. League Division 1,...,8,LCB,2022-11-01,33,10,1989,1,6,0,6


In [223]:
# Calculate age of players
df['club_joined'] = pd.to_datetime(df['club_joined'], format='%Y-%m-%d', errors='coerce')
df['today'] = pd.to_datetime('2022-11-01',format='%Y-%m-%d', errors='coerce')


df['years_inclub'] = ((df['today'] - df['club_joined']))
df['years_inclub'] = df['age'].astype(int)
df

# same as age, could just drop the column cause it says the same thing. 
# Could drop the club_joined column as well as it is not useful

,id,short_name,overall,potential,value_eur,wage_eur,height_cm,weight_kg,club_name,league_name,...,position,today,age,BirthMonth,BirthYear,body_type_encoded,work_rate_encoded,real_face_encoded,position_encoded,years_inclub
0,216302,E. García,71,71,1400000.0,10000,176,73,Club Atlético de San Luis,Mexican Liga MX,...,LB,2022-11-01,32,12,1989,4,6,0,5,32
1,237867,D. Cancola,65,71,1000000.0,2000,183,73,Ross County FC,Scottish Premiership,...,LDM,2022-11-01,26,10,1996,1,8,0,8,26
2,253472,E. Kahl,65,77,1600000.0,2000,178,69,Aarhus GF,Danish Superliga,...,LWB,2022-11-01,21,9,2001,4,2,0,13,21
3,223994,S. Mugoša,72,72,2300000.0,5000,188,81,Incheon United FC,Korean K League 1,...,LS,2022-11-01,30,2,1992,2,2,0,11,30
4,251635,A. Țigănașu,65,65,525000.0,3000,179,74,FC Botoşani,Romanian Liga I,...,LB,2022-11-01,32,6,1990,4,6,0,5,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6890,242007,C. Whelan,59,66,425000.0,2000,180,65,Carlisle United,English League Two,...,RCB,2022-11-01,24,12,1997,4,8,0,15,24
6891,187961,Paulinho,83,83,28500000.0,61000,183,80,Al Ahli,Saudi Abdul L. Jameel League,...,LCM,2022-11-01,34,7,1988,4,0,1,7,34
6892,257234,Y. Hamache,70,80,3400000.0,6000,177,73,Boavista FC,Portuguese Liga ZON SAGRES,...,LWB,2022-11-01,23,7,1999,4,2,0,13,23
6893,232511,S. Sasaki,71,71,1300000.0,7000,176,70,Sanfrecce Hiroshima,Japanese J. League Division 1,...,LCB,2022-11-01,33,10,1989,1,6,0,6,33


# Combining Features 

In [ ]:
def create_weighted_column(df, new_column, attributes, weights):
    """
    Create a new column as a weighted combination of specified attributes.

    Parameters:
    - df: DataFrame
    - new_column: str, the name of the new column to be created
    - attributes: list, the list of attribute columns to be combined
    - weights: list, the corresponding weights for each attribute

    Returns:
    - df: DataFrame, with the new column added
    """
    # Check if the length of attributes and weights match
    if len(attributes) != len(weights):
        raise ValueError("Length of 'attributes' and 'weights' must be the same.")

    # Normalize weights to ensure they sum up to 1
    weights = [w / sum(weights) for w in weights]

    # Create the new column by multiplying each attribute by its weight and summing them up
    df[new_column] = df[attributes].multiply(weights).sum(axis=1)

    return df

# Attributes all 
# Attacking 
attacking_attributes = ['attacking_crossing',  'attacking_finishing','attacking_heading_accuracy' ,'attacking_short_passing' ,'attacking_volleys']
weights_att = [0.2, 0.2,0.2,0.2,0.2]
# Movement
movement_attributes = ['movement_acceleration' , 'movement_sprint_speed' , 'movement_agility' , 'movement_reactions' , 'movement_balance']
weights_mov = [0.2, 0.2,0.2,0.2,0.2]
# Defensive 
defensive_attributes = ['defending_marking_awareness' , 'defending_standing_tackle' , 'defending_sliding_tackle' ]
weights_def = [1/3, 1/3, 1/3]
#Mentality 
mentality_attributes = ['mentality_aggression',  'mentality_interceptions','mentality_positioning' ,'mentality_vision' ,'mentality_penalties', 'mentality_composure']
weights_ment = [1/6, 1/6, 1/6, 1/6, 1/6,1/6]
#Skill 
skill_attributes = ['skill_dribbling' , 'skill_curve' , 'skill_fk_accuracy' , 'skill_long_passing' , 'skill_ball_control']
weights_skill = [0.2, 0.2,0.2,0.2,0.2]

# Combination added to dataframe
df = create_weighted_column(df, 'attacking_combined', attacking_attributes, weights_att)
df = create_weighted_column(df, 'movement_combined',  movement_attributes, weights_mov)
df = create_weighted_column(df, 'defensive_combined', defensive_attributes, weights_def)
df = create_weighted_column(df, 'mentality_combined', mentality_attributes, weights_ment)
df = create_weighted_column(df, 'skill_combined', skill_attributes, weights_skill)

# Display dataframe
df 